In [1]:
# Models
#from LoadData.models import Experiment, Sample, Dna, Vector, Measurement, Inducer, LoadProcess
# Data handling
import pandas as pd
import numpy as np
import json
import os
import openpyxl as opxl
from itertools import islice
import csv
import subprocess

In [26]:
def fix_synergy_time(df):
    t = np.array([])
    for i, value in enumerate(df['Time']):
        if i > 0:
            if df['Time'].iloc[i].hour < df['Time'].iloc[i-1].hour:
                t = np.append(t, [24 + value.hour + value.minute/60 + value.second/3600])
            else:
                t = np.append(t, [value.hour + value.minute/60 + value.second/3600])
        else:
            t = np.append(t, [value.hour + value.minute/60 + value.second/3600])
    df['Time'] = t

In [11]:
# Cargo Metadata desde el archivo JSON
data_str = open('MetaTest.json').read()
data = json.loads(data_str)
df_json = pd.read_json(data)
columns = [x+str(y) for x in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] for y in range(1,13)]
df_json.columns = columns
df_json.index = ['Strain', 'Media', 'DNA']
experiment_name = os.path.basename('ExpTest.xlsx').split('/')[-1].split('.')[0]

In [13]:
medidas = ['OD600:600', 'RFP-YFP:585/10,620/15', 'RFP-YFP:500/27,540/25', 'CFP:420/50,485/20', 'Results']

wb = opxl.load_workbook(filename = 'ExpTest.xlsx', data_only=True)
ws = wb['Data']
machine_name = ws['B'][8].value + str(ws['B'][9].value)

name_map = {'OD600:600':'OD', 'RFP-YFP:500/27,540/25':'YFP', 'CFP:420/50,485/20':'CFP',
'RFP-YFP:585/10,620/15':'RFP'}

In [18]:
# Cargo data
lista_rows = [(celda.value, celda.row, opxl.utils.column_index_from_string(celda.column))
        for celda in ws['A']
        if celda.value in medidas]
ws.delete_rows(0, lista_rows[0][1] + 1)
lista_rows2 = [(celda.value, celda.row, opxl.utils.column_index_from_string(celda.column))
        for celda in ws['A']
        if celda.value in medidas]

In [23]:
data = ws.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df = pd.DataFrame(data, columns=cols)
df = df.drop('T° OD600:600', axis=1)

In [27]:
df_OD = pd.DataFrame(df.iloc[0:lista_rows2[0][1] - 3])
df_OD['name'] = 'OD'
fix_synergy_time(df_OD)

,Time,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,H4,H5,H6,H7,H8,H9,H10,H11,H12,name
0,0.194444,0.082,0.083,0.083,0.082,0.082,0.082,0.083,0.083,0.084,...,0.082,0.084,0.082,0.087,0.082,0.082,0.083,0.08,0.083,OD
1,0.444444,0.082,0.083,0.084,0.082,0.083,0.082,0.083,0.083,0.084,...,0.082,0.084,0.082,0.087,0.082,0.082,0.083,0.08,0.083,OD
2,0.694444,0.082,0.084,0.084,0.082,0.083,0.083,0.084,0.084,0.085,...,0.082,0.084,0.082,0.088,0.083,0.082,0.083,0.081,0.084,OD
3,0.944444,0.082,0.085,0.084,0.082,0.084,0.083,0.085,0.084,0.087,...,0.083,0.084,0.082,0.089,0.083,0.083,0.083,0.081,0.084,OD
4,1.194444,0.082,0.085,0.085,0.082,0.084,0.084,0.086,0.085,0.088,...,0.083,0.084,0.082,0.09,0.083,0.083,0.083,0.081,0.084,OD
5,1.444444,0.081,0.086,0.086,0.083,0.085,0.085,0.087,0.086,0.09,...,0.083,0.084,0.082,0.09,0.084,0.083,0.083,0.081,0.085,OD
6,1.694444,0.081,0.088,0.087,0.083,0.085,0.086,0.089,0.087,0.094,...,0.083,0.084,0.082,0.09,0.084,0.083,0.084,0.082,0.085,OD
7,1.944444,0.081,0.089,0.088,0.084,0.086,0.088,0.091,0.089,0.098,...,0.083,0.085,0.083,0.09,0.084,0.084,0.084,0.082,0.085,OD
8,2.194444,0.081,0.092,0.09,0.084,0.088,0.09,0.094,0.092,0.104,...,0.083,0.085,0.083,0.09,0.085,0.084,0.085,0.083,0.085,OD
9,2.444444,0.081,0.095,0.092,0.085,0.09,0.092,0.097,0.096,0.11,...,0.084,0.085,0.083,0.087,0.085,0.085,0.085,0.083,0.085,OD


In [28]:
df_RFP = pd.DataFrame(df.iloc[lista_rows2[0][1] + 1:lista_rows2[1][1] - 3])
df_RFP['name'] = 'RFP'
fix_synergy_time(df_RFP)
df_RFP.index = range(97)
df_RFP

,Time,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,H4,H5,H6,H7,H8,H9,H10,H11,H12,name
0,0.206389,11043,11234,11400,11992,11411,11507,11619,11150,11440,...,12033,11605,11895,11506,11876,11864,11448,12266,11972,RFP
1,0.456389,11058,11138,11432,11940,11423,11557,11701,11193,11542,...,11990,11544,11880,11513,11830,11778,11416,12227,11906,RFP
2,0.706389,10968,11084,11441,11856,11441,11523,11730,11207,11588,...,11937,11518,11784,11409,11772,11716,11370,12130,11787,RFP
3,0.956389,10921,11062,11502,11805,11469,11580,11817,11217,11656,...,11901,11435,11789,11396,11730,11655,11400,12029,11746,RFP
4,1.206389,10881,10977,11526,11798,11480,11601,11932,11253,11697,...,11798,11427,11707,11353,11687,11635,11335,12019,11719,RFP
5,1.456389,10873,10988,11605,11805,11512,11665,12045,11293,11860,...,11768,11418,11707,11295,11702,11636,11347,11986,11737,RFP
6,1.706389,10851,10947,11703,11826,11622,11816,12230,11360,11980,...,11814,11415,11686,11341,11760,11632,11409,11994,11715,RFP
7,1.956389,10802,10906,11872,11885,11763,11961,12459,11473,12178,...,11817,11408,11744,11331,11738,11695,11370,11957,11681,RFP
8,2.206389,10829,10900,12040,11977,11925,12110,12735,11602,12438,...,11864,11409,11726,11286,11757,11702,11447,12069,11716,RFP
9,2.456389,10814,10860,12314,12056,12153,12423,13121,11812,12812,...,11841,11443,11743,11368,11804,11758,11507,12100,11744,RFP


In [29]:
df_YFP = pd.DataFrame(df.iloc[lista_rows2[1][1] + 1:lista_rows2[2][1] - 3])
df_YFP['name'] = 'YFP'
fix_synergy_time(df_YFP)
df_YFP.index = range(97)
df_YFP

,Time,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,H4,H5,H6,H7,H8,H9,H10,H11,H12,name
0,0.220278,3891,3895,4586,4417,4534,4625,5047,4352,4812,...,4571,4258,4417,4130,4337,4344,4222,4500,3996,YFP
1,0.470278,3842,3873,4739,4444,4657,4794,5324,4511,5089,...,4617,4278,4428,4122,4365,4369,4239,4459,3999,YFP
2,0.720278,3849,3823,4943,4470,4813,4960,5633,4668,5416,...,4662,4342,4481,4203,4390,4355,4251,4493,4007,YFP
3,0.970278,3831,3823,5146,4560,5042,5172,6009,4904,5826,...,4767,4395,4600,4227,4424,4346,4295,4464,3976,YFP
4,1.220278,3801,3810,5489,4672,5287,5481,6474,5211,6485,...,4913,4515,4680,4305,4445,4398,4296,4505,3988,YFP
5,1.470278,3809,3800,5900,4844,5683,5905,7234,5734,7397,...,5051,4654,4843,4371,4489,4434,4345,4590,4042,YFP
6,1.720278,3775,3797,6492,5078,6167,6456,8175,6361,8567,...,5306,4827,5040,4468,4529,4508,4412,4623,4037,YFP
7,1.970278,3819,3827,7250,5416,6826,7229,9475,7278,10240,...,5566,5035,5295,4633,4650,4588,4521,4741,4122,YFP
8,2.220278,3826,3839,8268,5757,7630,8201,10962,8363,12358,...,5888,5290,5469,4800,4739,4725,4616,4835,4180,YFP
9,2.470278,3796,3812,9492,6232,8734,9442,13009,9878,15092,...,6239,5576,5814,4942,4898,4833,4757,5006,4257,YFP


In [30]:
df_CFP = pd.DataFrame(df.iloc[lista_rows2[2][1] + 1:lista_rows2[3][1] - 3])
df_CFP['name'] = 'CFP'
fix_synergy_time(df_CFP)
df_CFP.index = range(97)
df_CFP

,Time,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,H4,H5,H6,H7,H8,H9,H10,H11,H12,name
0,0.234444,8041,7884,7683,7918,7789,7788,7720,7571,7402,...,8237,7751,7966,7949,7899,7958,7714,7941,5880,CFP
1,0.484444,7918,7713,7549,7751,7678,7674,7628,7442,7310,...,8054,7591,7795,7745,7744,7786,7573,7735,5757,CFP
2,0.734444,7894,7662,7548,7710,7604,7650,7686,7390,7298,...,8003,7605,7778,7736,7685,7706,7495,7768,5728,CFP
3,0.984444,7798,7653,7454,7660,7623,7600,7586,7378,7257,...,7915,7510,7670,7667,7605,7648,7477,7688,5705,CFP
4,1.234444,7740,7593,7490,7613,7586,7556,7593,7374,7314,...,7898,7483,7685,7646,7572,7613,7472,7627,5673,CFP
5,1.484444,7763,7595,7488,7594,7611,7625,7603,7394,7330,...,7915,7483,7648,7639,7570,7657,7440,7619,5683,CFP
6,1.734444,7735,7606,7532,7609,7626,7592,7630,7383,7396,...,7873,7478,7693,7648,7589,7650,7459,7627,5647,CFP
7,1.984444,7746,7629,7582,7643,7671,7628,7706,7459,7411,...,7882,7476,7670,7641,7626,7660,7470,7626,5677,CFP
8,2.234444,7736,7626,7593,7631,7705,7692,7756,7502,7558,...,7903,7457,7677,7596,7585,7651,7477,7646,5660,CFP
9,2.484444,7720,7634,7617,7626,7709,7715,7837,7522,7687,...,7886,7488,7699,7612,7602,7650,7481,7653,5672,CFP


In [ ]:
dfs = [df_OD, df_RFP, df_YFP, df_CFP]

# 1) Experiment
e = Experiment(name=experiment_name, machine=machine_name)
e.save()

# 2) Sample
# get experiment_id
# row, col, media, strain

# 3) DNA
# name

# Empiezo a recorrer los platillos que están en df_json: indexes: strain, media, dna:
for col_name, col_serie in df_json.iteritems():
# experiment_id
existing_dna = [i.name for i in Dna.objects.all()]

plate_row = col_name[0]
plate_col = col_name[1:]
st = col_serie['Strain']['value']
med = col_serie['Media']['value']

s = Sample(experiment=e, row=plate_row, col=plate_col, media=med, strain=st)
s.save()

DNA_name = col_serie['DNA']['value']
if DNA_name != 'None':
    if DNA_name not in existing_dna:
        d = Dna(name=DNA_name, sboluri='')
        d.save()
        v = Vector(dna=d, sample=s)
        v.save()
    else:
        d = Dna.objects.filter(name__exact=DNA_name)[0]
        v = Vector(dna=d, sample=s)
        v.save()
# 4) Measurement
# name, value, time
for df in dfs:
    for i, value in enumerate(df[col_name]):
        nam = df['name'].iloc[i]
        val = value
        #t = df['Time'].iloc[i]
        t = i
        m = Measurement(sample=s, name=nam, value=val, time=t)
        m.save()
